In [14]:
##Libraries##

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import insert
import pyodbc

In [43]:
##Connection to cmv_uat database##

server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [69]:
tradedate = 100627  ##Blotter + last 5 trades ###f"{sidechoice}" 
query = "SELECT * FROM [dbo].[fxblotter.t]"
# Read the SQL query into a DataFrame
fxblotter = pd.read_sql(query, connection, index_col='tradeid', parse_dates=True)
fxblotter.sort_index(inplace=True)
fxblotter['tradedate'] = pd.to_datetime(fxblotter['tradedate'])
fxblotter['valuedate'] = pd.to_datetime(fxblotter['valuedate'])
fxblottert = fxblotter.loc[fxblotter['clienteid'] == tradedate]
fxblottert.head(5)


,tiquetid,createdat,accionid,sideid,clienteid,comccyid,montocom,px_sp,venccyid,montoven,...,px_cover,monex_pxlast,monex_pp,estadoid,dealerid,productotipoid,solicitud,deletetradeid,UT1,UT2
tradeid,,,,,,,,,,,,,,,,,,,,,
202008140003542,3542,2020-08-14 12:00:00,1,1,100627,1,94.75,591.0,2,55997.25,...,596.20,596.20,596.20,1,2,1,0,NULL,None,None
202008190003525,3525,2020-08-19 10:54:00,1,1,100627,1,317.80,590.0,2,187502.00,...,595.91,595.91,595.91,1,2,1,0,NULL,None,None
202008240003496,3496,2020-08-24 10:40:00,1,1,100627,1,255.54,587.0,2,150001.98,...,595.05,595.05,595.05,1,2,1,0,NULL,None,None
202008270003460,3460,2020-08-27 10:10:00,1,1,100627,1,231.25,592.5,2,137015.63,...,596.20,596.20,596.20,1,2,1,0,NULL,None,None
202008280003455,3455,2020-08-28 08:58:00,1,1,100627,1,193.39,592.0,2,114486.88,...,595.91,595.91,595.91,1,2,1,0,NULL,None,None


In [72]:
### rate to monexpp

#forma 1
#filtro por cliente
#sumo todos los colones y todos los dolares, y calculo los dolares a tasa de cover 
bidfxblottert = fxblottert.loc[fxblotter['comccyid'] == 1 ]
askfxblottert = fxblottert.loc[fxblotter['venccyid'] == 1 ] 
#forma 2 
#diff px_sp y px_cover
spreadbidmpp =  bidfxblottert['monex_pp'] - fxblottert['px_sp'] 
spreadaskmpp =  askfxblottert['px_sp'] - fxblottert['monex_pp'] 
bidcount = fxblottert.loc[fxblotter['comccyid'] == 1, 'montocom'].count()
askcount = fxblottert.loc[fxblotter['venccyid'] == 1, 'montoven'].count()
spreadbidpxc =  bidfxblottert['px_cover'] - fxblottert['px_sp'] 
spreadaskpxc =  askfxblottert['px_sp'] - fxblottert['px_cover']
spreadbidpxl =  bidfxblottert['monex_pxlast'] - fxblottert['px_sp'] 
spreadaskpxl =  askfxblottert['px_sp'] - fxblottert['monex_pxlast']
bidmontoavg =  bidfxblottert.loc[fxblotter['comccyid'] == 1, 'montocom'].mean()
askmontoavg = askfxblottert.loc[fxblotter['venccyid'] == 1, 'montoven'].mean( )
##blendedspread = 


print1 ="Bid Spread to Monex P.P :  {:,} CRC"
print(print1.format(round(spreadbidmpp.mean(),3)))
print1 ="Bid Spread to Cover P.P :  {:,} CRC"
print(print1.format(round(spreadbidpxc.mean(),3)))
print1 ="Bid Spread to Monex last traded PX :  {:,} CRC"
print(print1.format(round(spreadbidpxl.mean(),3)))
print1 ="Bid monto promedio por transaccion :  {:,} USD"
print(print1.format(round(bidmontoavg,3)))

print1 ="Bids traded :  {:,} "
print(print1.format(bidcount))
print1 ="Ask Spread to Monex P.P :  {:,} CRC"
print(print1.format(round(spreadaskmpp.mean(),3)))
print1 ="Ask Spread to Cover P.P :  {:,} CRC"
print(print1.format(round(spreadaskpxc.mean(),3)))
print1 ="Ask Spread to Monex last traded PX :  {:,} CRC"
print(print1.format(round(spreadaskpxl.mean(),3)))
print1 ="Bid monto promedio por transaccion :  {:,} USD"
print(print1.format(round(askmontoavg,3)))

print1 ="Asks :  {:,}"
print(print1.format(askcount))

Bid Spread to Monex P.P :  3.984 CRC
Bid Spread to Cover P.P :  3.984 CRC
Bid Spread to Monex last traded PX :  3.984 CRC
Bid monto promedio por transaccion :  537.605 USD
Bids traded :  26 
Ask Spread to Monex P.P :  3.651 CRC
Ask Spread to Cover P.P :  3.651 CRC
Ask Spread to Monex last traded PX :  3.651 CRC
Bid monto promedio por transaccion :  7,790.04 USD
Asks :  8


In [37]:
###Rate to px_cover



p
print1 ="Bids traded :  {:,} "
print(print1.format(bidcount))
print1 ="Ask Spread to Cover P.P :  {:,} CRC"
print(print1.format(spreadaskmpp.mean()))
print1 ="Asks :  {:,}"
print(print1.format(askcount))




Bid Spread to Cover P.P :  1.8139999999999963 CRC
Bids traded :  24 
Ask Spread to Cover P.P :  -1.8139999999999963 CRC
Asks :  1


In [50]:
###Rate to px last
spreadbidpxl =  fxblottert['monex_pxlast'] - fxblottert['px_sp'] 
spreadaskpxl =  fxblottert['px_sp'] - fxblottert['monex_pxlast'] 



print1 ="Ask Spread to Monex last traded PX :  {:,} "
print(print1.format(spreadaskpxl.mean()))



Bid Spread to Monex last traded PX :  1.939999999999996 
Ask Spread to Monex last traded PX :  -1.939999999999996 


In [ ]:
## Print Report 




